In [ ]:
import numpy as np
def source_function( x, y ):
    if ( x >= 0.1 and x <= 0.3 and y >= 0.1 and y <= 0.3 ):
        return 1.0
    else:
        return 0.0

np_source_function = np.vectorize(source_function)

In [ ]:
# Finds the index k = 0,1,2,...,N-1 in the vector corresponding to the grid
# point (x_i, y_j) where i,j = 0,1,2,...,n-1

def indexFD(i, j, n):
  k = ( i*n + j )
  return k

def reverseIndexFD(k, n):
    i = k // n
    j = k % n
    return i, j

In [ ]:
import numpy as np

def boundaryConditions(n):

  extNodes = []

  # Find nodes at the boundary of the square
  for j in range(0,n):
    extNodes.append( indexFD(j,0,n) )
    extNodes.append( indexFD(j,n-1,n) )
    extNodes.append( indexFD(0,j,n) )
    extNodes.append( indexFD(n-1,j,n) )

  extNodes = np.unique(extNodes)
  intNodes = np.setdiff1d(np.arange(0,n**2), extNodes);

  return intNodes, extNodes

In [ ]:
import scipy as sp
import numpy as np

def diffusionMatrix( mux, muy, h ):
  n = int(1./h + 1)
  N = n**2
  A = sp.sparse.lil_array((N, N), dtype=np.float64)

  # Pass through all grid points and apply local stencil

  stencil = np.array([(mux+muy)*2., -mux*1., -mux*1., -muy*1., -muy*1.]) / h**2

  # Loop over each internal node in the grid, i,j = 1,2,...,n-2
  for i in range(1,n-1):
    for j in range(1,n-1):
      # Find k-indices of the four neighbouring nodes
      localStencilIndices = np.array([indexFD(i,j,n), indexFD(i+1,j,n), indexFD(i-1,j,n), indexFD(i,j+1,n), indexFD(i,j-1,n)])

      # Add the local stencil for node (x_i,y_j) to the matrix
      currentRow = indexFD(i,j,n)
      for m in range (0,5):
        A[currentRow, localStencilIndices[m]] = A[currentRow, localStencilIndices[m]] + stencil[m]

  # After the matrix A has been assembled, we convert it to the column-major format
  # for more efficient computations
  A = A.tocsc()

  return (A)

Check sparsity pattern of full matrix A and the factorised LU matrices.

In [ ]:
# #Check sparsity patterns
# h = 0.1
# mux = 0.1
# muy = 0.1
# n = int(1./h + 1)  # dimension of spatial mesh in each dimension
# N = n**2
# intNodes, extNodes = boundaryConditions(n)

# F = np.zeros((N, 1), dtype=np.float64)

# # Evaluate the source function
# for k in range (0,N):
#     i, j = reverseIndexFD(k, n)
#     F[k] = source_function( i*h, j*h )

# # Assemble the diffusion matrix
# A = diffusionMatrix(mux, muy, h)
# A_int = A[intNodes][:, intNodes]
# F_int = F[intNodes]

# print(f'Size of F_int: {F_int.shape}')
# print(f'Size of F: {F.shape}')

# # We plot the sparsity pattern of the matrix A to check that it has been
# # correctly assembled.
# #LU1 = sp.sparse.linalg.splu(A_int, 'NATURAL')
# LU1 = sp.sparse.linalg.spilu(A_int, drop_tol=10)
# L = LU1.L
# U = LU1.U
# L = L.toarray()
# U = U.toarray()
# #U = U.todense()
# #L = L.todense()

# fig, ax = plt.subplots(1, 3)
# ax1 = ax[0]
# ax2 = ax[1]
# ax3 = ax[2]


# cax1 = ax1.spy(A_int.toarray())
# ax1.set_title("Sparsity pattern")

# ax2.spy(U)
# ax2.set_title("(U)")

# ax3.spy(L)
# ax3.set_title("(L)")
# plt.show()




Check the speed without reoordering and with.

In [ ]:
# import time
# n = A_int.get_shape()[0]
# b = np.ones((n,1), dtype=float)

# tstart = time.thread_time()

# for i in range(0,1000):
#   #LU1 = sp.sparse.linalg.splu(A_int, 'NATURAL') # Compute A = LU without any reordering
#   LU1 = sp.sparse.linalg.spilu(A_int, drop_tol=10) # Compute A = LU without any reordering
#   LU1.solve(b)

# tend = time.thread_time()

# print('Mean time spent for one linear solve = ' + str((tend-tstart) * 10.) + ' milliseconds')

(d) incomplete-LU preconditioner

We continue to solve the system with the diffusion gradient, but now precondition the system using the incomplete-LU matrix.

In [ ]:
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.sparse.linalg import norm

def diffusionGradSolver( mux, muy, h, tol, maxiter, M ):
  n = int(1./h + 1)  # dimension of spatial mesh in each dimension
  N = n**2
  f = sp.sparse.csc_array((N, 1), dtype=np.float64)

  F = np.zeros((N, 1), dtype=np.float64)
  intNodes, extNodes = boundaryConditions(n)
  
  # Placeholder for an initial guess
  u0 = np.zeros((N, 1))

  # Evaluate the source function
  for k in range (0,N):
    i, j = reverseIndexFD(k, n)
    F[k] = source_function( i*h, j*h )
  
  # Assemble the diffusion matrix
  A = diffusionMatrix(mux, muy, h)
  A_int = A[intNodes][:, intNodes]
  F_int = F[intNodes]

  def iterate_count(x):
    iterate_count.iterate_counts += 1

  iterate_count.iterate_counts = 0

  f[intNodes], info = sp.sparse.linalg.cg(M, F_int, x0=None, tol=tol, maxiter=maxiter, M=None, callback=iterate_count, atol=None)
  print(f'Exit code: {info}')
  print(f'NUmber of iterations: {iterate_count.iterate_counts}')

  f[extNodes] = u0[extNodes]
  
  return f.toarray()


In [ ]:
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.sparse.linalg import norm

h = 0.02
mux = 1
muy = 1
droptol = 0.001

intNodes, extNodes = boundaryConditions(int(1./h+1))

#Preconditioner matrix
MA = diffusionMatrix(mux, muy, h)
MA = MA.todense()
MA_intNodes = MA[intNodes][:, intNodes]
M = sp.sparse.linalg.spilu(MA_intNodes, drop_tol=droptol)
L = M.L
U = M.U
#L = L.toarray()
#M = M.toarray()

M2 = L@U
M2 = M2.toarray()


In [ ]:
solgrad2 = diffusionGradSolver(1, 1, 0.02, 1e-6, 10000, M2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import matplotlib.patches as patches

fig, ax = plt.subplots()
hx = 0.1
# Make data.
X = np.arange(0., 1.+h, h)
Y = np.arange(0., 1.+h, h)
X, Y = np.meshgrid(X, Y)
Z = solgrad2.reshape(int(1./h+1),int(1./h+1))

# Plot the surface.
surf = ax.pcolor(X, Y, Z[:-1,:-1], shading='flat', cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Create a Rectangle patch
rect = patches.Rectangle((0.1, 0.1), 0.2, 0.2, linewidth=1, edgecolor='k', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)
hx = 0.1
# Add a grid
plt.grid(which='major', axis='both', linestyle='-', color='k', linewidth=0.5)
plt.minorticks_on()
plt.grid(which='minor', axis='both', linestyle='-', color='k', linewidth=0.5)
major_ticks = np.arange(0., 1.+hx, hx)
minor_ticks = np.arange(0., 1.+hx, hx)

plt.xticks(major_ticks)
plt.yticks(major_ticks)

plt.show()

(d) My PCG

In [ ]:
import scipy as sp
import numpy as np
from scipy.sparse.linalg import norm

def myPCG( A, b, L, U, tol, maxit ):
    #A is the matrix produced by diffusionMatrix
    #b is the RHS calculated by the source function
    #L and U are the factors of the preconditioner from spilu
    #tol is the tolerance for the residual
    # maxit is the maximum number of iterations
    N = A.shape[0]

    x0 = np.zeros((N, 1), dtype=np.float64)
    r0 = b - A@x0 #residual

    M = L@U
    z0 = sp.sparse.linalg.spsolve(M, r0)
    d0 = z0.copy() #preconditioned search direction
        
    x = x0.copy()
    r = r0.copy()
    z = z0.copy()
    d = d0.copy()
    
    alpha = 0
    beta = 0
       
    #print(f'i = {i}::::: Shape of alpha: {type(alpha)}, r0: {r0.shape}, z0: {z0.shape}, z: {z.shape}, d0: {d0.shape}, d: {d.shape}, A: {A.shape}, x0: {x0.shape}, x: {x.shape}, beta: {type(alpha)}')

    i = 0
    while i < maxit and np.linalg.norm(r) > tol:
        #print(alpha)
        
        alpha = (r0.T@z0)/(d0.T@(A@d0))
        x = x0 + alpha[0]*d
        print(f'x0: {x0.shape}')
        print(f'x1: {alpha[0,0]*d}')
        print(f'x: {x.shape}')
        r = r0 - alpha[0]*A@d0
        z = sp.sparse.linalg.spsolve(M, r)
        beta = (r.T@z)/(r0.T@z0)
        d0 = z + beta[0]*d0
        r0 = r
        x0 = x
        i += 1

    print(f'Converged in {i} iterations')
    print(f'Shape of x0: {x0.shape}')
    return x 

# def myPCG( A, b, L, U, tol, maxit ):
#     N = A.shape[0]
#     x0 = np.zeros((N, 1))
#     r0 = b - A @ x0
#     z0 = sp.sparse.linalg.spsolve(L @ U, r0)
#     d0 = z0.copy()
#     i = 0
#     while i < maxit and np.linalg.norm(r0) > tol:
#         Ad0 = A @ d0
#         alpha = (r0.T @ z0) / (d0.T @ Ad0)
#         x = x0 + alpha[0] * d0
#         r = r0 - alpha[0] * Ad0
#         z = sp.sparse.linalg.spsolve(L @ U, r)
#         beta = (r.T @ z) / (r0.T @ z0)
#         d0 = z + beta[0,0] * d0
#         r0 = r
#         x0 = x
#         print(f'i = {i}::::: Shape of alpha: {type(alpha)}, r0: {r0.shape}, z0: {z0.shape}, z: {z.shape}, d0: {d0.shape}, A: {A.shape}, x0: {x0.shape}, x: {x.shape}, beta: {type(alpha)}, Ad0: {Ad0.shape}')
#         i += 1

#     return x 

In [ ]:
import scipy as sp
import numpy as np
from scipy.sparse.linalg import norm

def myPCG2( A, b, L, U, tol, maxit ):
    #A is the matrix produced by diffusionMatrix
    #b is the RHS calculated by the source function
    #L and U are the factors of the preconditioner from spilu
    #tol is the tolerance for the residual
    # maxit is the maximum number of iterations
    N = A.shape[0]

    x0 = np.zeros((N,), dtype=np.float64)
    d0 = np.zeros((N,), dtype=np.float64)
    z0 = np.zeros((N,), dtype=np.float64)
    
    r0 = b - A@x0 #residual

    M = L@U
    z0 = sp.sparse.linalg.spsolve(M, r0)

    d0 = z0.copy() #preconditioned search direction
        
    x = x0.copy()
    r = r0.copy()
    z = z0.copy()
        
    alpha = 0
    beta = 0
       
    #print(f'i = {i}::::: Shape of alpha: {type(alpha)}, r0: {r0.shape}, z0: {z0.shape}, z: {z.shape}, d0: {d0.shape}, d: {d.shape}, A: {A.shape}, x0: {x0.shape}, x: {x.shape}, beta: {type(alpha)}')

    i = 0
    while i < maxit:
        #print(alpha)
        Top = (r0.T@z0)
        Bottom = (d0.T@(A@d0))
        alpha = Top/Bottom
        alphad = alpha*d0
        x = x0 + alphad
        r = r0 - alpha*A@d0

        if np.linalg.norm(r) < tol:
            break

        z = sp.sparse.linalg.spsolve(M, r)
        beta = (r.T@z)/(r0.T@z0)

        d0 = z + beta*d0
        r0 = r
        x0 = x
        print(f'Iteration = {i}')
        i += 1


    return x

In [ ]:
#Using the myPCG function
mux = 1
muy = 1
h = 0.02
n = int(1./h + 1)  # dimension of spatial mesh in each dimension
N = n**2
droptol = 0.1
tol = 0.1
maxit = 1000

A = diffusionMatrix(mux, muy, h)
intNodes, extNodes = boundaryConditions(n)
b = np.zeros((N,), dtype=np.float64)

# Evaluate the source function
for k in range (0,N):
    i, j = reverseIndexFD(k, n)
    b[k] = source_function( i*h, j*h )

A_int = A[intNodes][:, intNodes]
b_int = b[intNodes]

M = sp.sparse.linalg.spilu(A_int, drop_tol=droptol)

sol = sp.sparse.csc_array((N, 1), dtype=np.float64)

sol[intNodes] = myPCG2(A_int,b_int,M.L,M.U,tol,maxit)
sol[extNodes] = 0

print(f'shape of sol {sol.shape}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import matplotlib.patches as patches

fig, ax = plt.subplots()
sol = sol.todense()
# Make data.
X = np.arange(0., 1.+h, h)
Y = np.arange(0., 1.+h, h)
X, Y = np.meshgrid(X, Y)
Z = sol.reshape(int(1./h+1),int(1./h+1))

# Plot the surface.
surf = ax.pcolor(X, Y, Z[:-1,:-1], shading='flat', cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Create a Rectangle patch
rect = patches.Rectangle((0.1, 0.1), 0.2, 0.2, linewidth=1, edgecolor='k', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)
hx = 0.1
# Add a grid
plt.grid(which='major', axis='both', linestyle='-', color='k', linewidth=0.2)
plt.minorticks_on()
plt.grid(which='minor', axis='both', linestyle='-', color='k', linewidth=0.2)
major_ticks = np.arange(0., 1.+hx, hx)
minor_ticks = np.arange(0., 1.+hx, hx/5)

plt.xticks(major_ticks)
plt.yticks(major_ticks)
plt.show()

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse.linalg as spla

# Your provided functions: source_function, indexFD, reverseIndexFD, boundaryConditions, diffusionMatrix

# Initialize parameters
h = 0.02
mux = 1
muy = 1
n = int(1./h + 1)
N = n**2
drop_tol = 0.1
intNodes, extNodes = boundaryConditions(n)
# Generate diffusion matrix
A = diffusionMatrix(mux, muy, h)
A_int = A[intNodes][:, intNodes]

# Create ILU preconditioner
Mx = spla.spilu(A_int.tocsc(), drop_tol=drop_tol)
#Mx = spla.LinearOperator(A_int.shape, ilu.solve)

# Define source term and initialize vectors
b = np.array([source_function(*reverseIndexFD(i, n)) for i in range(N)])
b_int = b[intNodes]
x = np.zeros_like(b_int)  # Initial guess
r = b_int - A_int.dot(x)  # Initial residual
z = Mx@r  # Apply preconditioner
p = z.copy()
tol = 1e-8  # Convergence tolerance
maxiter = 1000  # Maximum number of iterations

# Start iterations
for iteration in range(maxiter):
    Ap = A_int.dot(p)
    r_dot_z = np.dot(r, z)
    alpha = r_dot_z / np.dot(p, Ap)
    x += alpha * p
    r_new = r - alpha * Ap
    z_new = Mx@r_new

    # Check for convergence
    if np.linalg.norm(r_new) < tol:
        print(f"Solution converged in {iteration} iterations.")
        break

    beta = np.dot(r_new, z_new) / r_dot_z
    p = z_new + beta * p
    r = r_new
    z = z_new

# Check if solution did not converge within max iterations
if np.linalg.norm(r_new) >= tol:
    print("Solution did not converge.")

# x now contains the solution